# Central Alps

We want to cover the rectangle Basel-Bard-Trento-Innsbruck
aka `[[[7.734,45.583], [11.249,45.583], [11.249,47.517], [7.734, 47.517], [7.734,45.583]]]`.

That includes
* all Switzerland south-easy of Basel
* small part of Aosta and Piemonte
* all Lombardia north of Milano
* tiny piece of Trentino
* half of Austria's Tyrol, west of Innsbruck

### Choice of bounding-box
A choice was made to have both Central Alps and Eastern Alps as simple rectangles, while keeping data small.
This means we keep either Italy's *Occidental Dolomites* ([c2c](https://www.camptocamp.org/outings?qa=draft,great&bbox=1243491,5721172,1310793,5796316&act=skitouring)) or Germany's *Lechtal Alps* ([c2c](https://www.camptocamp.org/outings?qa=draft,great&bbox=1285592,6022754,1352894,6097898&act=skitouring)).
The bbox with Occidental Dolomites is `[[[7.734,45.583], [11.953,45.583], [11.953,47.517], [7.734, 47.517], [7.734,45.583]]]`.
But it makes less sense in terms of mapping to to massifs, and in terms of relative size for West/Central/East.

# Central Alps - Switzerland download

This notebook outputs file `data/AlpsC-ch-slopes.tif`, obtained from SwissTopo APIs.

The bounds requested are Basel-Bard-Trento-Innsbruck
aka `[[[7.734,45.583], [11.249,45.583], [11.249,47.517], [7.734, 47.517], [7.734,45.583]]]`

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Preamble

import logging
logging.basicConfig(level=logging.DEBUG)

from dataclasses import astuple

import os
import sys
sys.path.append(os.path.realpath('./src'))
from bbox import BBox

# Switzerland

We get the data using the awesome *Spatial Temporal Asset Catalog* geodata APIs documented at [geo.admin.ch](http://geo.admin.ch/en/geo-services/geo-services/download-services/stac-api.html) and [data.geo.admin.ch](https://data.geo.admin.ch/api/stac/static/spec/v0.9/api.html#operation/getFeatures)

for documentation:, here's the description of the dataset (`data_desc` below):
```json
{'stac_version': '0.9.0',
   'id': 'ch.swisstopo.swissalti3d',
   'title': 'swissALTI3D',
   'description': 'swissALTI3D is an extremely precise digital elevation model which describes the surface of Switzerland and the Principality of Liechtenstein without vegetation and development. It is updated in an cycle of 6 years.',
   'summaries': {'proj:epsg': [2056], 'eo:gsd': [0.5, 2.0]},
   'extent': {
       'spatial': {'bbox': [[5.950, 45.721, 10.500, 47.821]]},
       'temporal': {'interval': [['2019-01-01T00:00:00Z', '2021-01-01T00:00:00Z']]}},
   'providers': [{
       'name': 'Federal Office of Topography - swisstopo',
       'roles': ['producer', 'licensor'],
       'url': 'https://www.swisstopo.admin.ch'}],
   'license': 'proprietary',
   'created': '2021-02-05T13:54:56.380277Z',
   'updated': '2021-09-20T11:58:15.865192Z',
   'links' : [{'rel': 'self', 'href': '...'}, ...]
}
```


In [47]:
import json
from urllib.request import urlopen
from urllib.parse import urlencode #, quote_plus

def apiget(url, **params):
    if params:
        if not url.endswith('?'): url += '?'
        url += urlencode(params) #, quote_via=quote_plus)
    print('GET', url)
    return json.loads(urlopen(url).read())

In [44]:
# Based on this we selct in the full list of URLS downloaded from

# from urllib.parse import urlencode, quote_plus

# params = {
#     'format': 'image/tiff; application=geotiff; profile=cloud-optimized',
#     'resolution': 2,
#     'srid': 2056,
#     'state': 'current',
#     'csv': True
# }
# resp = urlopen(
#         'https://ogd.swisstopo.admin.ch/services/swiseld/services/assets/ch.swisstopo.swissalti3d/search?'
#         + urlencode(params, quote_via=quote_plus)).read()
# import json
# csvurl = json.loads(resp)['href']
# # 'https://ogd.swisstopo.admin.ch/resources/ch.swisstopo.swissalti3d-Mktp9SsL.csv'
# csvcontent = urlopen(csvurl).read()

#2622 1262

In [45]:
# Get all swiss datasets
collections = apiget("https://data.geo.admin.ch/api/stac/v0.9/collections")

GET https://data.geo.admin.ch/api/stac/v0.9/collections


In [48]:
# Helper
def get_first_link(data, rel):
    return [lnk for lnk in data['links'] if lnk['rel'] == rel][0]['href']
# Get the first 100 of features of swissalti3d
data_desc = [c for c in collections['collections'] if c['id'] == 'ch.swisstopo.swissalti3d'][0]
items_link = get_first_link(data_desc, 'items')
items = apiget(items_link, bbox=','.join(map(str,astuple(bbcalps)))) #, limit=100 == default == max

GET https://data.geo.admin.ch/api/stac/v0.9/collections/ch.swisstopo.swissalti3d/items?bbox=7.734%2C45.583%2C11.249%2C47.517&limit=100


In [51]:
# Explore a sample
sample = items['features'][0]
print(len(items['features']),
    sample['id'],
    sample['bbox'],
    list(sample['assets']),
    sep='\n')

def only_two_meters_tif(feature):
    return [a['href'] for a  in feature['assets'].values() if a['eo:gsd'] == 2 and a['href'].endswith('.tif')]

only_two_meters_tif(sample)

100
swissalti3d_2019_2622-1085
[7.7221803, 45.9162116, 7.7351165, 45.92524]
['swissalti3d_2019_2622-1085_0.5_2056_5728.tif', 'swissalti3d_2019_2622-1085_0.5_2056_5728.xyz.zip', 'swissalti3d_2019_2622-1085_2_2056_5728.tif', 'swissalti3d_2019_2622-1085_2_2056_5728.xyz.zip']


['https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2019_2622-1085/swissalti3d_2019_2622-1085_2_2056_5728.tif']

In [ ]:
features = []
while True:
    features += items['features']
    if len(features) % 10000 == 0:
        print(len(features))
    try:
        items = apiget(get_first_link(items, 'next'))
    except IndexError:
        break
# Takes ~200 seconds & iterations
print(len(features))  #26064

In [53]:
features_keep = features

In [54]:
urls = [url
    for feat in features
    for url in only_two_meters_tif(feat)
] #if Bbox(*feat['bbox']).intersect(bbcalps)]  # <- covered by the bbox parameter
len(urls)

26064

In [55]:
url = urls[0]
url

'https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2019_2622-1085/swissalti3d_2019_2622-1085_2_2056_5728.tif'

In [61]:
from urllib.request import urlretrieve
import os

folder = 'data/chc/dwn'
os.makedirs(folder, exist_ok=True)
for cururl in urls:
    destpath = '/'.join((folder, cururl.split('/')[-1]))
    if not os.path.exists(destpath):
        urlretrieve(url, destpath)

# follow download of 26064 files in eslope/development/data/ch/
# at around 3 files/second it takes >2h
# feel free to symlink in case of disk space issue

In [60]:
# print('''{
#   "type": "FeatureCollection",
#   "features": [''')
# for bb in (bbcalps, *(Bbox(*feat['bbox']) for feat in items['features'])):
#     print('''    {   "type": "Feature",
#         "geometry": { "type": "LineString",
#             "coordinates": [ [%s, %s], [%s, %s] ] },
#         "properties": {}
#     },''' % (bb.w, bb.s, bb.e, bb.n))
# print(''']}''')

In [64]:
%cd $folder
basenames = [url.split('/')[-1] + '\n' for url in urls]
vrtpieces = 'vrt_input_files.txt'
vrt = 'AlpsC-ch-dem.vrt'
with open(vrtpieces, 'w') as f:
    f.writelines(basenames)
!wc -l $vrtpieces

!gdalbuildvrt -input_file_list $vrtpieces $vrt
%cd ..
g_zstd='-co bigtiff=yes -co compress=zstd -co predictor=2 -co zstd_level=1 -co sparse_ok=true'
!gdaldem slope $=g_zstd dwn/$vrt AlpsC-ch-slopes.tif
%cd ../..

/home/me/code/eddy-geek/eslope/development/data/ch/dwn
26064 vrt_input_files.txt
0...10...20...30...40...50...60...70...80...90...100 - done.
/home/me/code/eddy-geek/eslope/development/data/ch
0...10.